In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy import units

import importlib

import simulation
import snapshot_obj
import simulation_tracing
import dataset_compute
import subhalo
import halo_matching

In [ ]:
importlib.reload(simulation)
importlib.reload(snapshot_obj)
importlib.reload(simulation_tracing)
importlib.reload(dataset_compute)
importlib.reload(subhalo)
importlib.reload(halo_matching)

In [ ]:
data = {"plain-LCDM" : {},
        "curv-p082" : {}
       }

snap_end = 127
snap_start = 101
matcher = halo_matching.SnapshotMatcher()

id_z0 = {"plain-LCDM" : {"M31" : (1,0), "MW" : (2,0)},
         "curv-p082" : {"M31" : (1,0), "MW" : (1,1)}
        }

for sim_name, sim_id in zip(data.keys(), ["V1_LR_fix", "V1_LR_curvaton_p082_fix"]):
    sim = simulation.Simulation(sim_id)
    data[sim_name]["Redshift"] = sim.get_redshifts(snap_start, snap_end+1)

    # Build merger tree and trace centrals:
    mtree = simulation_tracing.MergerTree(sim, matcher, branching="BackwardBranching")
    mtree.build_tree(snap_start, snap_end+1)
    
    m31 = subhalo.SubhaloTracer(sim, snap_end, id_z0[sim_name]["M31"][0],
                                id_z0[sim_name]["M31"][1])
    mw = subhalo.SubhaloTracer(sim, snap_end, id_z0[sim_name]["MW"][0],
                                id_z0[sim_name]["MW"][1])
    m31.trace(mtree)
    mw.trace(mtree)

    data[sim_name]["M31_mass"] = m31.get_halo_data("MassType", snap_start, snap_end+1) \
        * units.g.to(units.Msun)
    data[sim_name]["MW_mass"] = mw.get_halo_data("MassType", snap_start, snap_end+1) \
        * units.g.to(units.Msun)
    data[sim_name]["Separation"] = mw.distance_to_central(m31, snap_start, snap_end+1) \
        * units.cm.to(units.kpc)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharey="row", sharex="col",
                         figsize=(10,8))

plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.04)

axes[0,0].set_yscale('log')
axes[1,0].set_yscale('log')

axes[0,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,1].yaxis.set_ticks_position('both')
axes[2,0].invert_xaxis()
axes[2,1].invert_xaxis()

axes[2,0].set_xlabel("Redshift")
axes[2,1].set_xlabel("Redshift")
axes[0,0].set_ylabel('$M_\mathrm{M31} [\mathrm{M_\odot}]$')
axes[1,0].set_ylabel('$M_\mathrm{MW} [\mathrm{M_\odot}]$')
axes[2,0].set_ylabel('Distance [kpc]')

for i, sim_name in enumerate(data.keys()):
    axes[0,i].set_title(sim_name)
    
    m = data[sim_name]["M31_mass"]
    z = data[sim_name]["Redshift"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[0, i].plot(z, m[:,pt], c=col, label=lab)
    
    m = data[sim_name]["MW_mass"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[1, i].plot(z, m[:,pt], c=col, label=lab)
        
    axes[2, i].plot(z, np.linalg.norm(data[sim_name]["Separation"], axis=1))
    

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharey="row", sharex="col",
                         figsize=(10,8))


plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.04)

axes[0,0].set_yscale('log')
axes[1,0].set_yscale('log')

axes[0,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,1].yaxis.set_ticks_position('both')
axes[2,0].invert_xaxis()
axes[2,1].invert_xaxis()

axes[2,0].set_xlabel("Redshift")
axes[2,1].set_xlabel("Redshift")
axes[0,0].set_ylabel('$M_\mathrm{M31} [\mathrm{M_\odot}]$')
axes[1,0].set_ylabel('$M_\mathrm{MW} [\mathrm{M_\odot}]$')
axes[2,0].set_ylabel('Distance [kpc]')

for i, sim_name in enumerate(data.keys()):
    axes[0,i].set_title(sim_name)
    
    m = data[sim_name]["M31_mass"]
    z = data[sim_name]["Redshift"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[0, i].plot(z, m[:,pt], c=col, label=lab)
    
    m = data[sim_name]["MW_mass"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[1, i].plot(z, m[:,pt], c=col, label=lab)
        
    axes[2, i].plot(z, np.linalg.norm(data[sim_name]["Separation"], axis=1))
    


In [ ]:
fig = plt.figure(figsize=(9, 9), constrained_layout=True)

gs0 = fig.add_gridspec(2, 1, height_ratios=[3,1])

# Add space between gs[0] and gs[1]:
gsx = gs0[0].subgridspec(2,1, height_ratios=[25,1])

gs1 = gsx[0].subgridspec(3, 2, wspace=0.03, hspace=0.01)

ax = np.empty((3,2), dtype=object)
ax[0,0] = fig.add_subplot(gs1[0,0])
ax[1,0] = fig.add_subplot(gs1[1,0], sharex=ax[0, 0], sharey=ax[0,0])
ax[2,0] = fig.add_subplot(gs1[2,0], sharex=ax[1, 0])
plt.setp(ax[0,0].get_xticklabels(), visible=False)
plt.setp(ax[1,0].get_xticklabels(), visible=False)
    
ax[0,1] = fig.add_subplot(gs1[0,1], sharey=ax[0,0])
ax[1,1] = fig.add_subplot(gs1[1,1], sharex=ax[0, 1], sharey=ax[1,0])
ax[2,1] = fig.add_subplot(gs1[2,1], sharex=ax[1, 1], sharey=ax[2,0])
plt.setp(ax[0,1].get_xticklabels(), visible=False)
plt.setp(ax[0,1].get_yticklabels(), visible=False)
plt.setp(ax[1,1].get_xticklabels(), visible=False)
plt.setp(ax[1,1].get_yticklabels(), visible=False)
plt.setp(ax[2,1].get_yticklabels(), visible=False)

ax[0,0].set_yscale('log')
ax[1,0].set_yscale('log')

ax[0,0].yaxis.set_ticks_position('both')
ax[1,0].yaxis.set_ticks_position('both')
ax[1,0].yaxis.set_ticks_position('both')
ax[1,1].yaxis.set_ticks_position('both')
ax[2,0].invert_xaxis()
ax[2,1].invert_xaxis()

ax[2,0].set_xlabel("Redshift")
ax[2,1].set_xlabel("Redshift")
ax[0,0].set_ylabel('$M_\mathrm{M31} [\mathrm{M_\odot}]$')
ax[1,0].set_ylabel('$M_\mathrm{MW} [\mathrm{M_\odot}]$')
ax[2,0].set_ylabel('Distance [kpc]')



gs3 = gs0[1].subgridspec(1, 4, wspace=0.08, hspace=0)
ax2 = np.empty(4, dtype=object)
ax2[0] = fig.add_subplot(gs3[0])
ax2[1] = fig.add_subplot(gs3[1], sharey=ax2[0])
plt.setp(ax2[1].get_yticklabels(), visible=False)

ax2[2] = fig.add_subplot(gs3[2], sharex=ax2[0], sharey=ax2[0])
ax2[3] = fig.add_subplot(gs3[3], sharex=ax2[1], sharey=ax2[0])
plt.setp(ax2[2].get_yticklabels(), visible=False)
plt.setp(ax2[3].get_yticklabels(), visible=False)

ax2[0].set_ylabel('$z$ [kpc]')
ax2[0].set_xlabel('$x$ [kpc]')
ax2[1].set_xlabel('$y$ [kpc]')
ax2[2].set_xlabel('$x$ [kpc]')
ax2[3].set_xlabel('$y$ [kpc]')

for i, sim_name in enumerate(data.keys()):
    ax[0,i].set_title(sim_name)   
    
    m = data[sim_name]["M31_mass"]
    z = data[sim_name]["Redshift"]
    r = data[sim_name]["Separation"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        ax[0, i].plot(z, m[:,pt], c=col, label=lab)
    
    m = data[sim_name]["MW_mass"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        ax[1, i].plot(z, m[:,pt], c=col, label=lab)
        
    ax[2, i].plot(z, np.linalg.norm(r, axis=1))


    ax2[2*i].plot(r[:,0], r[:,2], 'b')
    ax2[2*i].plot(r[0,0], r[0,2], 'ro', label='earliest')
    ax2[2*i+1].plot(r[:,1], r[:,2], 'b')
    ax2[2*i+1].plot(r[0,1], r[0,2], 'ro', label='earliest')

In [ ]:
m = data[sim_name]["M31_mass"]
z = data[sim_name]["Redshift"]
for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
    ax00.plot(z, m[:,pt], c=col, label=lab)

ax01 = fig.add_subplot(gs1[0,1], sharey=ax00)
plt.setp(ax01.get_yticklabels(), visible=False)
m = data[sim_name]["M31_mass"]
z = data[sim_name]["Redshift"]
for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
    ax01.plot(z, m[:,pt], c=col, label=lab)
    
    

In [ ]:
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.04)

axes[0,0].set_yscale('log')
axes[1,0].set_yscale('log')

axes[0,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,0].yaxis.set_ticks_position('both')
axes[1,1].yaxis.set_ticks_position('both')
axes[2,0].invert_xaxis()
axes[2,1].invert_xaxis()

axes[2,0].set_xlabel("Redshift")
axes[2,1].set_xlabel("Redshift")
axes[0,0].set_ylabel('$M_\mathrm{M31} [\mathrm{M_\odot}]$')
axes[1,0].set_ylabel('$M_\mathrm{MW} [\mathrm{M_\odot}]$')
axes[2,0].set_ylabel('Distance [kpc]')

for i, sim_name in enumerate(data.keys()):
    axes[0,i].set_title(sim_name)
    
    m = data[sim_name]["M31_mass"]
    z = data[sim_name]["Redshift"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[0, i].plot(z, m[:,pt], c=col, label=lab)
    
    m = data[sim_name]["MW_mass"]
    for pt, lab, col in zip([0,1,4],['gas', 'dm', 'stars'], ['red', 'green', 'blue']):
        axes[1, i].plot(z, m[:,pt], c=col, label=lab)
        
    axes[2, i].plot(z, np.linalg.norm(data[sim_name]["Separation"], axis=1))
    

In [ ]:
ax[1].set_xlabel('$z$', fontsize=16)

ax[0].plot(d[:,0], d[:,1], 'b')
ax[0].plot(d[:,0], d[:,1], 'ko')
ax[0].plot(d[0,0], d[0,1], 'ro', label='earliest')
ax[0].set_xlabel('$x$ [kpc]', fontsize=16)
ax[0].set_ylabel('$y$ [kpc]', fontsize=16)

ax[1].plot(d[:,0], d[:,2], 'b')
ax[1].plot(d[:,0], d[:,2], 'ko')
ax[1].plot(d[0,0], d[0,2], 'ro')
ax[1].set_xlabel('$x$ [kpc]', fontsize=16)
ax[1].set_ylabel('$z$ [kpc]', fontsize=16)

ax[0].legend()

# Save figure:

filename = 'satellite_trajectory_gn{}sgn{}_{}.png'.format(gn,sgn,sim.sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures",sim.sim_id)
print(path)

# If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
#plt.savefig(os.path.join(path,filename), dpi=200)

# Plot evolution of mass components

In [ ]:
sat_snaps = np.nonzero([inst is not None for inst in satellite.tracer])[0]
cen_snaps = np.nonzero([inst is not None for inst in central.tracer])[0]
snap_start = max(min(sat_snaps), min(cen_snaps))
snap_stop = min(max(sat_snaps), max(cen_snaps)) + 1
print(snap_start, snap_stop)

In [ ]:
z = sim.get_redshifts(snap_start, snap_stop)
masses = satellite.get_halo_data("MassType") * units.g.to(units.Msun)
print(z)

In [ ]:
masses = satellite.get_halo_data("MassType") * units.g.to(units.Msun)

In [ ]:
fig,ax = plt.subplots(nrows=2, figsize=(10,8))

ax[0].set_title('gn{} sgn{}'.format(gn,sgn))

ax[0].set_yscale('log')
ax[0].set_ylabel('Mass $[\mathrm{M_\odot}]$', fontsize=16)

ax[1].set_xlabel('$z$', fontsize=16)
ax[1].set_ylabel('$r$ [kpc]', fontsize=16)

ax[0].yaxis.set_ticks_position('both')
ax[1].yaxis.set_ticks_position('both')

ax[0].invert_xaxis()
ax[1].invert_xaxis()

for pt,l in zip([0,1,4],['gas', 'dm', 'stars']):
    ax[0].plot(z, masses[:,pt], label=l)
    
ax[0].legend()

ax[1].plot(z, np.linalg.norm(d, axis=1))

# Save figure:

filename = 'satellite_mass_evol_gn{}sgn{}_{}.png'.format(gn,sgn,sim.sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures",sim.sim_id)

# If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
#plt.savefig(os.path.join(path,filename), dpi=200)